
# DNCformer: Parallel-Enrichment Transformer–DNC (Notebook Prototype)

Implements a **parallel enrichment** architecture that adds a **Transformer-style DNC block** alongside a standard Transformer block, with a learned **gating** to mix their outputs. It wires on top of a **frozen ~4B LLM** (Phi-3-mini-4k-instruct by default), and provides **lightweight train/eval** loops and **unit-like tests**.

**Hardware:** designed for a single GPU (e.g., RTX 3090 24GB) using AMP (`bf16` if available, otherwise `fp16`).  
**Structure:** Config → Utils → DNC Memory → Transformer Controller → DNCformer Block → Parallel Enrichment → Frozen Base + N Blocks → Data → Train → Eval → Tests.


## 0. Imports, config, and environment

In [20]:
import sys, platform, torch
from dataclasses import dataclass
from typing import Optional, List, Tuple, Dict, Any
import os, math, random, time, numpy as np
from torch import nn, Tensor
from torch.nn import functional as F
import contextlib
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch.optim.lr_scheduler import LambdaLR
from typing import List, Optional


print("python exe:", sys.executable)
print("torch file:", torch.__file__)
print("torch ver:", torch.__version__, "| torch.version.cuda:", torch.version.cuda)
print("cuda available:", torch.cuda.is_available(), "| device count:", torch.cuda.device_count())


python exe: C:\Users\andre\miniconda3\envs\dncformer\python.exe
torch file: C:\Users\andre\miniconda3\envs\dncformer\lib\site-packages\torch\__init__.py
torch ver: 2.5.1 | torch.version.cuda: 12.6
cuda available: True | device count: 1


In [21]:
# --- Configuration ---
@dataclass
class Config:
    base_model_id: str = "microsoft/Phi-3-mini-4k-instruct"  # ~3.8B
    d_model: Optional[int] = None          # If None, infer from base model hidden size
    n_blocks: int = 2                      # number of parallel enrichment blocks
    attn_heads: int = 8                    # heads in DNC controller
    attn_dropout: float = 0.1
    ffn_mult: float = 4.0
    dnc_read_heads: int = 2
    dnc_cell_size: int = 64                # memory slot width
    dnc_nr_cells: int = 256                # number of memory slots
    gate_bias_init: float = -1.0           # bias to prefer transformer at init
    lr: float = 2e-4                       
    weight_decay: float = 0.01
    max_seq_len: int = 1024                # training seq length
    train_steps: int = 200                 # small sanity pass
    warmup_steps: int = 20
    grad_clip: float = 1.0
    precision: str = "bf16"                # "bf16" | "fp16" | "fp32"
    use_torch_compile: bool = False
    device: str = "cuda"
    log_every: int = 10
    batch_size: int = 8

CFG = Config()

seed = 42
random.seed(seed); torch.manual_seed(seed)

device = torch.device(CFG.device if torch.cuda.is_available() else "cpu")
print("Device:", device, "CUDA:", torch.cuda.is_available())

amp_dtype = None
if CFG.precision == "bf16" and torch.cuda.is_available() and torch.cuda.is_bf16_supported():
    amp_dtype = torch.bfloat16
elif CFG.precision == "fp16" and torch.cuda.is_available():
    amp_dtype = torch.float16
else:
    amp_dtype = torch.float32

print("AMP dtype:", amp_dtype)

Device: cuda CUDA: True
AMP dtype: torch.bfloat16


In [22]:
# --- Config patch toggles ---
try:
    CFG
except NameError:
    class _Tmp: pass
    CFG = _Tmp()
# safe defaults if not already present
if not hasattr(CFG, 'batch_size'): CFG.batch_size = 8
if not hasattr(CFG, 'gate_reg_lambda'): CFG.gate_reg_lambda = 0.0   # only applied on memory-tagged batches
if not hasattr(CFG, 'hist_every'): CFG.hist_every = 200             # histogram cadence
if not hasattr(CFG, 'force_g'): CFG.force_g = None                  # None, or 0.0 or 1.0
if not hasattr(CFG, 'gate_temp'): CFG.gate_temp = 1.0


In [23]:
# --- SDPA selection (prefer PyTorch SDPA; avoid flash-attn) ---

def sdpa_ctx():
    """Return a fresh attention-kernel selection context each time it's called.
    Uses PyTorch SDPA (math + mem-efficient) and disables flash-attn to avoid warnings.
    """
    try:
        from torch.backends.cuda import sdp_kernel  # callable context manager in PyTorch 2.x
        return sdp_kernel(enable_flash=False, enable_math=True, enable_mem_efficient=True)
    except Exception:
        return contextlib.nullcontext()


## 1. Utilities

#### 1.1 Model Information and factory

In [24]:
_GLOBAL = {}

def causal_mask(sz: int, device=None):
    return torch.full((sz, sz), float("-inf"), device=device).triu(1)

def count_params(model: nn.Module):
    return sum(p.numel() for p in model.parameters())

def requires_grad_(module: nn.Module, flag: bool):
    for p in module.parameters():
        p.requires_grad_(flag)
    return module

def print_shapes(**tensors):
    for k, v in tensors.items():
        if isinstance(v, (list, tuple)):
            print(k, [tuple(x.shape) for x in v])
        elif isinstance(v, torch.Tensor):
            print(k, tuple(v.shape))
        else:
            print(k, type(v))


#### 1.2 Gate metrics utils (mean, frac>0.5, entropy)

In [25]:
def _reduce_gate_tensor(g: torch.Tensor) -> torch.Tensor:
    # g: (B,T,*) -> (B,T)
    if g.dim() == 3:
        return g.mean(dim=-1)
    return g

@torch.no_grad()
def _gate_metrics(g: torch.Tensor):
    """
    Returns scalar (mean, frac>0.5, entropy)
    """
    g2 = _reduce_gate_tensor(g.detach())
    mean_val = float(g2.mean().item())
    frac = float((g2 > 0.5).float().mean().item())
    eps = 1e-6
    p = g2.clamp(eps, 1 - eps)
    # binary entropy (natural log base)
    ent = float((-(p * (p + eps).log() + (1 - p) * (1 - p + eps).log())).mean().item())
    return mean_val, frac, ent

#### 1.3 Memory Freeing/handling

In [26]:
def free_head_and_cache():
    # delete typical globals and clear allocator
    with contextlib.suppress(Exception):
        del globals()['head']
    with contextlib.suppress(Exception):
        del globals()['tok']
    gc.collect(); torch.cuda.empty_cache()
    cuda_report("after free")

#### 1.4 Environment Export

In [27]:
# --- Optional: export environment specs (run in the dncformer conda env) ---
import shutil, subprocess, sys
def _run(cmd):
    try:
        print(">", cmd); subprocess.run(cmd, shell=True, check=True)
    except Exception as e:
        print("Command failed (may be expected on some setups):", e)
        
def export_current_env():
    print("Exporting conda environment and pip freeze to current working directory...")
    _run("conda env export --from-history > environment.yml")
    _run("conda env export > environment.lock.yml")
    _run("python -m pip list --format=freeze > requirements-pip.txt")
    print("Done. If any commands failed, run them in your terminal inside the active conda env.")

## 2. Models

#### 2.1: DNC Memory

In [28]:
class DNCMemory(nn.Module):
    """
    Compact DNC memory:
    - memory M: (B, N, W)
    - usage u: (B, N)
    - link L: (B, N, N) temporal links
    - precedence p: (B, N)
    - read weights rw: (B, R, N)
    - write weights ww: (B, N)
    - read vectors r: (B, R, W)
    """
    def __init__(self, nr_cells: int, cell_size: int, read_heads: int):
        super().__init__()
        self.probe = None  # optional callable to record per-step state
        self.N = nr_cells
        self.W = cell_size
        self.R = read_heads

    def reset(self, B: int, device=None):
        device = device or next(self.parameters(), torch.empty(0, device="cpu")).device
        M = torch.zeros(B, self.N, self.W, device=device)
        u = torch.zeros(B, self.N, device=device)
        L = torch.zeros(B, self.N, self.N, device=device)
        p = torch.zeros(B, self.N, device=device)
        rw = F.one_hot(torch.zeros(B, self.R, dtype=torch.long, device=device), num_classes=self.N).float()
        r = torch.zeros(B, self.R, self.W, device=device)
        return {"M": M, "u": u, "L": L, "p": p, "rw": rw, "r": r}

    @staticmethod
    def _cosine_sim(M: torch.Tensor, k: torch.Tensor, eps=1e-6):
        # M: (B, N, W), k: (B, W) or (B, R, W)
        if k.dim() == 2:
            k = k.unsqueeze(1)  # (B,1,W)
        B, R, W = k.shape
        Mnorm = F.normalize(M, p=2, dim=-1)
        knorm = F.normalize(k, p=2, dim=-1)
        sim = torch.einsum("bnw,brw->brn", Mnorm, knorm)  # (B, R, N)
        return sim

    def _allocation(self, u: torch.Tensor):
        # u: (B, N) in [0,1]
        δ = 1e-6
        u = δ + (1 - δ) * u                 # avoid tiny values before cumprod
        B, N = u.shape
        # sort ascending usage -> free list φ
        sorted_u, phi = torch.sort(u, dim=-1, descending=False)  # (B,N)
        # exclusive cumprod of sorted_u
        ones = torch.ones(B, 1, device=u.device, dtype=u.dtype)
        prod_excl = torch.cumprod(torch.cat([ones, sorted_u], dim=1), dim=1)[:, :-1]  # (B,N)
        a_sorted = (1 - sorted_u) * prod_excl                                        # (B,N)
        # invert the sort to original order
        inv_phi = torch.argsort(phi, dim=-1)
        a = a_sorted.gather(1, inv_phi)                                              # (B,N)
        return a


    def forward(self, x_if: dict, state: dict):
        """
        x_if (interface dict) must contain:
        - k_read: (B,R,W), beta_read: (B,R,1)
        - k_write: (B,W), beta_write:(B,1)
        - erase: (B,W) in (0,1), write_vec: (B,W)
        - free_gates: (B,R,1) in (0,1), alloc_gate:(B,1), write_gate:(B,1) in (0,1)
        - read_mode: (B,R,3) softmax over {backward, content, forward}
        """
        M, u, L, p, rw, r = state["M"], state["u"], state["L"], state["p"], state["rw"], state["r"]
        B, N, W = M.shape

        # --- Usage update (faithful, stable) ---
        # previous write weights; keep as (B,1,N) for easy broadcasting
        ww_prev = state.get("ww", torch.zeros(M.size(0), 1, self.N, device=M.device, dtype=M.dtype))
        # writes increase usage
        u = u + (1 - u) * (1 - torch.prod(1 - ww_prev, dim=1))   # -> (B,N)
        # free gates release usage at read locations (per-location retention)
        psi = torch.prod(1 - x_if["free_gates"] * rw, dim=1)     # (B,N), since free_gates:(B,R,1), rw:(B,R,N)
        u = torch.clamp(u * psi, 0, 1)


        # 2.1) Write weighting (robust broadcasting) ---
        # sim_w: (B,1,N) if k_write=(B,W); (B,R,N) if k_write=(B,R,W)
        sim_w = self._cosine_sim(M, x_if["k_write"])
        # beta_w: expect (B,1) or (B,R,1); make sure it has the trailing head axis
        beta_w = x_if["beta_write"]
        if beta_w.dim() == 2:           # (B,1) or (B,R) -> add trailing axis
            beta_w = beta_w.unsqueeze(-1)  # -> (B,1,1) or (B,R,1)
        # content weights over memory locations
        cw = F.softmax(sim_w * beta_w, dim=-1)  # (B,1,N) or (B,R,N)
        # canonical DNC: single write head; if multiple heads exist, reduce over heads
        if cw.size(1) > 1:
            cw = cw.mean(dim=1)                # -> (B,N)  (alternatives: sum or a learned reduce)
        else:
            cw = cw.squeeze(1)                 # -> (B,N)
        # allocation weights from usage (B,N)
        a = self._allocation(u)                # (B,N)
        # interpolate content vs allocation via alloc_gate, then apply write_gate
        alloc = x_if["alloc_gate"]             # (B,1)
        write_gate = x_if["write_gate"]        # (B,1)
        # Broadcast (B,1) over N
        ww = write_gate * (alloc * a + (1.0 - alloc) * cw)  # -> (B,N) via broadcasting
        state["ww"] = ww
        
        # 2.2) save current ww as "previous write weights" for t+1
        state["ww_prev"] = ww.unsqueeze(1)   # keep grads for BPTT; use .detach() only if you explicitly want to stop gradients across steps

        # 3) Memory write
        erase = x_if["erase"].unsqueeze(1)  # (B,1,W)
        write_vec = x_if["write_vec"].unsqueeze(1)  # (B,1,W)
        M = M * (1 - ww.unsqueeze(-1) * erase) + ww.unsqueeze(-1) * write_vec

        # 4) Temporal link
        prev_p = p
        p = (1 - ww.sum(dim=-1, keepdim=True)) * p + ww  # precedence
        L = (1 - ww.unsqueeze(2) - ww.unsqueeze(1)) * L + torch.einsum("bn,bm->bnm", prev_p, ww)
        L = L * (1 - torch.eye(N, device=M.device).unsqueeze(0))

        # 5) Read weighting
        cr = F.softmax(self._cosine_sim(M, x_if["k_read"]) * x_if["beta_read"], dim=-1)  # (B,R,N)
        fwd = torch.einsum("brn,bnm->brm", rw, L)       # (B,R,N) forward
        bwd = torch.einsum("brn,bmn->brm", rw, L)       # (B,R,N) backward
        read_mode = F.softmax(x_if["read_mode"], dim=-1)  # (B,R,3)
        rw = read_mode[:,:,0:1]*bwd + read_mode[:,:,1:2]*cr + read_mode[:,:,2:3]*fwd
        r = torch.einsum("brn,bnw->brw", rw, M)  # (B,R,W)

        state = {"M": M, "u": u, "L": L, "p": p, "rw": rw, "r": r}
        # aggregated lightweight stats (per-step)
        try:
            _stats = {}
            with torch.no_grad():
                _stats["u_mean"] = state["u"].mean().detach()
                try:
                    _stats["M_norm_mean"] = state["M"].norm(dim=-1).mean().detach()
                except Exception:
                    pass
                try:
                    _stats["rw_max_mean"] = rw.max(dim=-1).values.mean().detach()
                except Exception:
                    pass
                try:
                    _stats["ww_max_mean"] = ww.max(dim=-1).values.mean().detach()
                except Exception:
                    pass
            state["stats"] = _stats
        except Exception:
            pass
        
        if self.probe is not None:
            M = state["M"].detach().float().cpu()
            u = state["u"].detach().float().cpu()
            L = state["L"].detach().float().cpu()
            rw_cpu = rw.detach().float().cpu()
            ww_cpu = state.get("ww", torch.zeros_like(state["u"])).detach().float().cpu()
            self.probe(
                {
                "u": u,                         # (B,N)
                "ww": ww_cpu,                   # (B,N)
                "rw": rw_cpu,                   # (B,R,N)
                "M_norm": M.norm(dim=-1),       # (B,N)
                "L_diag_mean": torch.diagonal(L, dim1=-2, dim2=-1).mean(dim=-1), # (B,)
                }
            )
        return r, state


#### 2.2 Transformer-style Controller (sequence mode)

In [29]:
class TransformerController(nn.Module):
    """
    Lightweight Transformer encoder producing the DNC interface vector.
    Inputs: X (B, T, d_in); prev_reads typically concatenated to X before calling.
    """
    def __init__(self, d_in: int, d_model: int, heads: int, dropout: float=0.1, ffn_mult: float=4.0):
        super().__init__()
        self.proj_in = nn.Linear(d_in, d_model)
        self.ln1 = nn.LayerNorm(d_model)
        self.attn = nn.MultiheadAttention(d_model, heads, dropout=dropout, batch_first=True)
        self.ln2 = nn.LayerNorm(d_model)
        self.ff = nn.Sequential(
            nn.Linear(d_model, int(d_model*ffn_mult)),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(int(d_model*ffn_mult), d_model),
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, x: torch.Tensor, attn_mask: Optional[torch.Tensor]=None) -> torch.Tensor:
        h = self.proj_in(x)
        h = self.ln1(h)
        attn_out, _ = self.attn(h, h, h, attn_mask=attn_mask, need_weights=False)
        h = h + self.dropout(attn_out)
        h = self.ln2(h)
        h2 = self.ff(h)
        h = h + self.dropout(h2)
        return h

#### 2.3 DNCformer Block (controller → interface → memory)

In [30]:
class DNCInterfaceHead(nn.Module):
    """
    Projects controller hidden to DNC interface:
    read keys (R*W), read strengths (R), write key (W), write strength (1),
    erase (W in (0,1)), write vector (W), free_gates (R in (0,1)),
    alloc_gate (1), write_gate (1), read_mode (R*3 softmax).
    """
    def __init__(self, d_model: int, R: int, W: int):
        super().__init__()
        self.R, self.W = R, W
        out = R*W + R + W + 1 + W + W + R + 1 + 1 + R*3
        self.proj = nn.Linear(d_model, out)

    def forward(self, h: torch.Tensor):
        B, T, D = h.shape
        v = self.proj(h)  # (B,T,out)
        idx = 0
        def take(sz): 
            nonlocal idx
            part = v[..., idx:idx+sz]; idx += sz; return part
        R, W = self.R, self.W
        k_read = take(R*W).view(B,T,R,W)
        beta_read = F.softplus(take(R)).view(B,T,R,1)
        k_write = take(W).view(B,T,W)
        beta_write = F.softplus(take(1)).view(B,T,1)
        erase = torch.sigmoid(take(W)).view(B,T,W)
        write_vec = take(W).view(B,T,W)
        free_gates = torch.sigmoid(take(R)).view(B,T,R,1)
        alloc_gate = torch.sigmoid(take(1)).view(B,T,1)
        write_gate = torch.sigmoid(take(1)).view(B,T,1)
        read_mode = take(R*3).view(B,T,R,3)
        return {
            "k_read": k_read, "beta_read": beta_read,
            "k_write": k_write, "beta_write": beta_write,
            "erase": erase, "write_vec": write_vec,
            "free_gates": free_gates, "alloc_gate": alloc_gate,
            "write_gate": write_gate, "read_mode": read_mode
        }

class DNCformerBlock(nn.Module):
    def __init__(self, d_in: int, d_model: int, R: int, W: int, N: int, heads: int, dropout: float, ffn_mult: float):
        super().__init__()
        self.R, self.W, self.N = R, W, N
        self.ctrl = TransformerController(d_in + R*W, d_model, heads=heads, dropout=dropout, ffn_mult=ffn_mult)
        self.if_head = DNCInterfaceHead(d_model, R=R, W=W)
        self.mem = DNCMemory(nr_cells=N, cell_size=W, read_heads=R)
        self.out_proj = nn.Linear(d_model + R*W, d_model)  # fuse controller + reads

    def forward(self, x: torch.Tensor, state: Optional[dict]=None):
        # x: (B,T,d_in); state carries memory fields; if None -> reset
        B, T, D = x.shape
        if state is None:
            state = self.mem.reset(B, device=x.device)
        reads = state["r"].reshape(B, self.R*self.W)  # (B,RW)
        reads_seq = reads.unsqueeze(1).expand(B, T, self.R*self.W)
        ctrl_in = torch.cat([x, reads_seq], dim=-1)  # concat
        h = self.ctrl(ctrl_in, attn_mask=causal_mask(T, device=x.device))  # (B,T,d_model)

        # step over time for memory I/O
        r_list = []
        new_state = state
        iface = self.if_head(h)
        for t in range(T):
            x_if = {k: v[:,t] for k,v in iface.items()}
            r_t, new_state = self.mem(x_if, new_state)
            r_list.append(r_t)
        Rseq = torch.stack(r_list, dim=1)  # (B,T,R,W)
        reads_flat = Rseq.reshape(B,T,self.R*self.W)
        fused = torch.cat([h, reads_flat], dim=-1)
        y = self.out_proj(fused)  # (B,T,d_model)
        return y, new_state


#### 2.4 Parallel Enrichment Block (Transformer path ‖ DNCformer path + gating)

In [31]:
class VanillaTransformerBlock(nn.Module):
    def __init__(self, d_model: int, heads: int, dropout: float=0.1, ffn_mult: float=4.0):
        super().__init__()
        self.collect_metrics = False
        self.ln1 = nn.LayerNorm(d_model)
        self.attn = nn.MultiheadAttention(d_model, heads, dropout=dropout, batch_first=True)
        self.ln2 = nn.LayerNorm(d_model)
        self.ff = nn.Sequential(
            nn.Linear(d_model, int(d_model*ffn_mult)),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(int(d_model*ffn_mult), d_model),
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, x: torch.Tensor, attn_mask: Optional[torch.Tensor]=None, gate_override: None = None):
        h = self.ln1(x)
        a, _ = self.attn(h, h, h, attn_mask=attn_mask, need_weights=False)
        h = x + self.dropout(a)
        z = self.ln2(h)
        z2 = self.ff(z)
        return h + self.dropout(z2)

class ParallelEnrichmentBlock(nn.Module):
    def __init__(self, d_model: int, d_in: int, R: int, W: int, N: int, heads: int, dropout: float, ffn_mult: float, gate_bias_init: float):
        super().__init__()
        self.collect_metrics = False
        self.vanilla = VanillaTransformerBlock(d_model, heads, dropout, ffn_mult)
        self.dncblock = DNCformerBlock(d_in=d_in, d_model=d_model, R=R, W=W, N=N, heads=heads, dropout=dropout, ffn_mult=ffn_mult)
        self.pre_gate_ln = nn.LayerNorm(2*d_model)
        self.gate = nn.Linear(2*d_model, d_model)
        nn.init.constant_(self.gate.bias, gate_bias_init)

    def forward(self, x: torch.Tensor, dnc_state=None, gate_override: None = None):
        # Both branches consume the same x (B,T,d_model) and produce (B,T,d_model)
        T = x.size(1)
        mask = causal_mask(T, device=x.device)
        vt = self.vanilla(x, attn_mask=mask)
        dt, dnc_state = self.dncblock(x, state=dnc_state)
        z = torch.cat([vt, dt], dim=-1)
        g = torch.sigmoid(self.gate(self.pre_gate_ln(z)))
        out = g*dt + (1-g)*vt
        
        # collect per-block metrics if requested
        if self.collect_metrics:
            try:
                import math, torch as _t
                eps = 1e-6
                g_clamp = g.clamp(min=eps, max=1-eps)
                g_entropy = (-(g_clamp*_t.log(g_clamp) + (1-g_clamp)*_t.log(1-g_clamp))).mean()
                vt_norm = vt.norm(dim=-1).mean()
                dt_norm = dt.norm(dim=-1).mean()
                _stats = dnc_state.get("stats", {}) if isinstance(dnc_state, dict) else {}
                # ensure CPU-detached tiny tensors
                self.last_metrics = {
                    "g_mean": g.mean().detach(),
                    "g_entropy": g_entropy.detach(),
                    "vt_norm": vt_norm.detach(),
                    "dt_norm": dt_norm.detach(),
                    **_stats
                }
            except Exception:
                self.last_metrics = None
        else:
            self.last_metrics = None
            return out, dnc_state, g 


#### 2.5. Frozen Base LLM + N Enrichment Blocks

In [32]:
def load_base_model(model_id: str):
    tok = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        torch_dtype=(torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16) if amp_dtype!=torch.float32 else None,
        device_map=None,
        trust_remote_code=True,
        attn_implementation="sdpa",
    ).to("cuda")
    if tok.pad_token is None:
        tok.pad_token = tok.eos_token
    requires_grad_(model, False)
    model.config.output_hidden_states = True
    model.config.use_cache = False
    return tok, model

class DNCFormerHead(nn.Module):
    def __init__(self, base: AutoModelForCausalLM, cfg):
        super().__init__()
        self.base = base
        d_model = base.config.hidden_size if cfg.d_model is None else cfg.d_model
        self.d_model = d_model
        self.blocks = nn.ModuleList([
            ParallelEnrichmentBlock(
                d_model=d_model, d_in=d_model,
                R=cfg.dnc_read_heads, W=cfg.dnc_cell_size, N=cfg.dnc_nr_cells,
                heads=cfg.attn_heads, dropout=cfg.attn_dropout,
                ffn_mult=cfg.ffn_mult, gate_bias_init=cfg.gate_bias_init
            ) for _ in range(cfg.n_blocks)
        ])
        self.proj_out = nn.Identity()

    def forward(self, input_ids: torch.Tensor, attention_mask: Optional[torch.Tensor]=None):
        with torch.no_grad():
            with sdpa_ctx():
                out = self.base(input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True, use_cache=False)
        h = out.hidden_states[-1]  # (B,T,d_model)
        dnc_states = [None]*len(self.blocks)
        gates = []
        for i, blk in enumerate(self.blocks):
            h, dnc_states[i], g = blk(h, dnc_state=dnc_states[i])
            gates.append(g.detach())
        logits = self.base.lm_head(self.proj_out(h).to(self.base.lm_head.weight.dtype))
        return logits, gates



    def forward_with_metrics(self, input_ids: torch.Tensor, attention_mask: "Optional[torch.Tensor]" = None,
                             gate_override: "Optional[float]" = None):
        with torch.no_grad():
            with sdpa_ctx():
                out = self.base(input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True, use_cache=False)
        h = out.hidden_states[-1].to(device)  # ensure CUDA
        dnc_states = [None]*len(self.blocks)
        gates_det = []
        gates_raw = []
        per_block = []
        # enable metrics collection per block
        for blk in self.blocks:
            if hasattr(blk, "collect_metrics"):
                blk.collect_metrics = True
        for i, blk in enumerate(self.blocks):
            h, dnc_states[i], g = blk(h, dnc_state=dnc_states[i], gate_override=gate_override)
            gates_raw.append(g)
            gates_det.append(g.detach())
            if hasattr(blk, "last_metrics") and blk.last_metrics is not None:
                per_block.append(blk.last_metrics)
            else:
                per_block.append({})
            # reset flag to avoid overhead elsewhere
            if hasattr(blk, "collect_metrics"):
                blk.collect_metrics = False
        # lm head on its own device, then back to CUDA
        lm_dev = self.base.lm_head.weight.device
        y = self.proj_out(h).to(lm_dev, dtype=self.base.lm_head.weight.dtype)
        logits = self.base.lm_head(y).to(device)
        aux = {"per_block": per_block, "gates_raw": gates_raw, "gates_detached": gates_det}
        aux["g_entropy_block"] = []
        for g in gates_det:
            _, _, ent = _gate_metrics(g)
            aux["g_entropy_block"].append(ent)
        return logits, gates_det, aux


## 3. Data Generation

#### 3.1 Synthetic tasks + simple instruction-following

In [33]:
def make_copy_task(batch, T, vocab=50):
    x = torch.randint(5, vocab, (batch, T))
    return x

def make_reverse_task(batch, T, vocab=50):
    x = torch.randint(5, vocab, (batch, T))
    y = torch.flip(x, dims=[1])
    return x, y

def make_needle_task(batch, T, needle_len=5, vocab=100):
    x = torch.randint(5, vocab, (batch, T))
    for b in range(batch):
        start = random.randint(0, T-needle_len-5)
        needle = torch.randint(5, vocab, (needle_len,))
        x[b, start:start+needle_len] = needle
        x[b, -1] = needle[0]
    return x

INSTR_PAIRS = [
    ("Reverse the string: abcd", "dcba"),
    ("Add two numbers: 7 + 12", "19"),
    ("Instruction: say hello", "hello"),
    ("Uppercase this: cat", "CAT"),
]

def tokenize_instruction_pairs(tok, pairs, max_len):
    texts = [f"### Instruction:\n{p}\n\n### Response:\n" for p,_ in pairs]
    labels = [ans for _,ans in pairs]
    input_ids = tok(texts, return_tensors="pt", padding=True, truncation=True, max_length=max_len).input_ids
    label_ids = tok(labels, return_tensors="pt", padding=True, truncation=True, max_length=max_len).input_ids
    # naive pack: just use inputs; real packing/labels can be elaborated
    return input_ids, label_ids

def make_repeat_copy(batch: int, T: int, repeat_min=2, repeat_max=4, vocab=100, pad_id: int = 0, device: str = "cpu") -> torch.Tensor:
    L = max(1, T // 2)
    x = torch.randint(1, vocab, (batch, L), device=device, dtype=torch.long)
    r = torch.randint(repeat_min, repeat_max + 1, (batch,), device=device)
    out = torch.full((batch, T), pad_id, dtype=torch.long, device=device)
    for i in range(batch):
        seq = x[i].repeat_interleave(int(r[i].item()))
        out[i, :min(T, seq.numel())] = seq[:T]
    return out

def make_n_back(batch: int, T: int, n: int = 3, vocab=50) -> torch.Tensor:
    return torch.randint(1, vocab, (batch, T))

def format_instruction(tok, instr: str, resp: str, max_len=256) -> torch.Tensor:
    prompt = f"### Instruction:\n{instr}\n\n### Response:\n{resp}"
    return tok(prompt, return_tensors="pt", truncation=True, max_length=max_len).input_ids[0]


#### 3.2 HF dataset integration

In [34]:
def hf_instruction_loader(dataset_name="tatsu-lab/alpaca", split="train", text_field=("instruction","output"), max_items=5000):
    try:
        from datasets import load_dataset
    except Exception:
        print("Install 'datasets' to enable HF loading: pip install datasets -q")
        return []
    ds = load_dataset(dataset_name, split=split)
    pairs = []
    i_field, o_field = text_field
    for ex in ds:
        instr = ex.get(i_field, ""); out = ex.get(o_field, "")
        if instr and out: pairs.append((instr, out))
        if len(pairs) >= max_items: break
    random.shuffle(pairs); return pairs

def make_hf_batch(tok, pairs: List[Tuple[str,str]], batch: int, max_len=256) -> torch.Tensor:
    if not pairs:
        return torch.full((batch, max_len), tok.pad_token_id, dtype=torch.long)
    batch_ids = []
    for _ in range(batch):
        instr, out = random.choice(pairs)
        ids = format_instruction(tok, instr, out, max_len=max_len)
        batch_ids.append(ids)
    maxL = min(max(x.size(0) for x in batch_ids), max_len)
    out_ids = torch.full((batch, maxL), tok.pad_token_id, dtype=torch.long)
    for i, ids in enumerate(batch_ids):
        ids = ids[:maxL]; out_ids[i, :ids.size(0)] = ids
    return out_ids


#### 3.3 Haystack batch creation

In [35]:
# --- Haystack (needle) eval: long-span retrieval of a key's value ---
def make_haystack_batch(batch: int, T: int = 256, vocab: int = 1024, sentinel: int = 3):
    """
    Build sequences: ... K V ... K ?  (next token should be V)
    Returns: input_ids [B,T], answer_ids [B], query_pos [B] (position of '?')
    """
    assert T >= 12, "T too small for haystack layout"
    x = torch.randint(5, vocab, (batch, T), dtype=torch.long)
    K = torch.randint(5, vocab, (batch,), dtype=torch.long)
    V = torch.randint(5, vocab, (batch,), dtype=torch.long)

    # Place (K,V) in the first half
    p1 = torch.randint(low=T//8, high=T//2 - 2, size=(batch,))
    x[torch.arange(batch), p1] = K
    x[torch.arange(batch), p1 + 1] = V

    # Place (K, sentinel) in the last quarter
    p2 = torch.randint(low=3*T//4, high=T - 2, size=(batch,))
    x[torch.arange(batch), p2] = K
    x[torch.arange(batch), p2 + 1] = sentinel  # '?'
    query_pos = p2 + 1  # position of '?'; we will look at logits at this position

    return x, V, query_pos

#### 3.4 MixtureSampler

In [36]:
class MixtureSampler:
    def __init__(self, gens: List, weights: List[float], names: Optional[List[str]] = None):
        self.gens = gens
        import torch as _t
        self.weights = list(map(float, weights))
        self.p = _t.tensor(self.weights, dtype=_t.float32)  # CPU is fine for multinomial
        self.p /= (self.p.sum() + 1e-8)
        self.names = names if names is not None else [f"g{i}" for i in range(len(gens))]
        self.last_name = None

    def __call__(self, batch: int) -> torch.Tensor:
        import torch as _t
        idx = _t.multinomial(self.p, 1).item()
        self.last_name = self.names[idx]
        return self.gens[idx](batch)
    
    def set_weights(self, weights):
        """Update mixture probabilities at runtime (used by schedules)."""
        import torch as _t
        ws = list(map(float, weights))
        t = _t.tensor(ws, dtype=_t.float32, device=self.p.device)
        s = float(t.sum().item())
        if s <= 0:
            raise ValueError("Mixture weights must sum to > 0")
        self.p = t / s
        self.weights = ws
        # Optional sanity: warn if names length mismatches weights
        if hasattr(self, "names") and len(self.names) != len(ws):
            print(f"[MixtureSampler] Warning: len(names)={len(self.names)} != len(weights)={len(ws)}")

#### 3.5 Mixture Builder

In [37]:
def _build_mixer(tok, weights, hf_dataset="tatsu-lab/alpaca", hf_max_items=2000) -> MixtureSampler:
    """
    Build a mixture of generators: [HF, copy, repeat, nback].
    - If hf_dataset is Alpaca-like (has instruction/output), we use hf_instruction_loader + make_hf_batch.
    - Else we treat it as text-only (e.g., roneneldan/TinyStories), tokenizing and making windowed sequences.
    - If HF fails/empty, we drop it and renormalize over synthetics.
    """
    mx = int(getattr(CFG, "max_seq_len", 256))
    pad_id = getattr(tok, "pad_token_id", 0) or 0

    gens, wts, names = [], [], []

    hf_ok = False
    hf_reason = ""
    gen_hf = None

    if hf_dataset:
        try:
            # Heuristic: use instruction loader for alpaca-like IDs
            if "alpaca" in hf_dataset.lower():
                pairs = hf_instruction_loader(hf_dataset, "train", ("instruction", "output"),
                                              max_items=hf_max_items)
                if pairs:
                    def gen_hf(b): 
                        return make_hf_batch(tok, pairs, b, max_len=mx)
                    hf_ok = True
                else:
                    hf_reason = "hf_instruction_loader returned 0 pairs"
            else:
                # Text-only fallback (e.g., roneneldan/TinyStories)
                from datasets import load_dataset
                # Try streaming first, then non-streaming
                try:
                    ds = load_dataset(hf_dataset, split="train", streaming=True)
                except Exception:
                    ds = load_dataset(hf_dataset, split="train")

                # Find a usable text key
                text_key = None
                common_keys = ("text", "content", "story", "document", "body", "article")

                feats = getattr(ds, "features", None)
                if feats:
                    for k in common_keys:
                        if k in feats:
                            text_key = k
                            break

                if text_key is None:
                    # Probe first example (works for streaming iterable)
                    try:
                        first_ex = next(iter(ds))
                        for k in common_keys:
                            if k in first_ex:
                                text_key = k
                                break
                    except StopIteration:
                        pass

                if text_key is None:
                    hf_reason = "no usable text field (tried: %s)" % ",".join(common_keys)
                else:
                    import random as _rnd
                    import torch
                    samples = []
                    for ex in ds:
                        txt = ex.get(text_key, None)
                        if not txt:
                            continue
                        ids = tok(txt, return_tensors="pt", add_special_tokens=False).input_ids.squeeze(0)
                        if ids.numel() < 8:
                            continue
                        samples.append(ids.cpu())
                        if len(samples) >= int(hf_max_items):
                            break

                    if len(samples) == 0:
                        hf_reason = "collected 0 tokenized samples"
                    else:
                        def gen_hf(b: int) -> torch.Tensor:
                            out = []
                            for _ in range(b):
                                ids = _rnd.choice(samples)
                                n = ids.numel()
                                if n >= mx:
                                    s = _rnd.randint(0, n - mx)
                                    seq = ids[s:s+mx]
                                else:
                                    pad = torch.full((mx - n,), pad_id, dtype=torch.long)
                                    seq = torch.cat([ids, pad], dim=0)
                                out.append(seq.unsqueeze(0))
                            return torch.cat(out, dim=0)
                        hf_ok = True
        except Exception as e:
            hf_reason = f"{type(e).__name__}: {e}"

    # Assemble mixture in the agreed order (hf, copy, repeat, nback)
    if hf_ok and gen_hf is not None:
        gens.append(gen_hf); wts.append(weights[0]); names.append("hf")
        s_w = list(weights[1:])
    else:
        if hf_dataset:
            print(f"HF dataset unavailable or empty; using synthetic only. reason={hf_reason}")
        s_w = list(weights[1:])  # keep caller's relative weights for synthetics

    # Synthetic tasks (your existing closures)
    def gen_copy(b):   return make_copy_task(b, T=min(mx, 128), vocab=100)
    def gen_repeat(b): return make_repeat_copy(b, T=min(mx, 128), vocab=100, pad_id=pad_id, device="cpu")
    def gen_nback(b):  return make_n_back(b, T=min(mx, 128), n=5, vocab=50)

    gens.extend([gen_copy, gen_repeat, gen_nback])
    wts.extend(s_w)
    names.extend(["copy", "repeat", "nback"])

    return MixtureSampler(gens, wts, names=names)


## 4 Logging utilities

#### 4.1 TensorBoard Logger

In [38]:
import os, time, json
try:
    from torch.utils.tensorboard import SummaryWriter
    TB_AVAILABLE = True
except Exception as e:
    print("TensorBoard not available:", e)
    TB_AVAILABLE = False

class TBLogger:
    def __init__(self, logdir: Optional[str] = None, run_name: Optional[str] = None):
        self.enabled = TB_AVAILABLE
        self.writer = None
        if not self.enabled:
            return
        logdir = logdir or "./runs"
        run_name = run_name or time.strftime("dncformer-%Y%m%d-%H%M%S")
        self.path = os.path.join(logdir, run_name)
        os.makedirs(self.path, exist_ok=True)
        self.writer = SummaryWriter(self.path)

    def log_scalars(self, step: int, loss: float, lr: float, gate_means: List[float]):
        if not (self.enabled and self.writer): return
        self.writer.add_scalar("train/loss", loss, step)
        self.writer.add_scalar("train/lr", lr, step)
        for i, gm in enumerate(gate_means):
            self.writer.add_scalar(f"gates/block_{i}_mean", gm, step)

    def add_image_hw(self, tag: str, img_hw: "torch.Tensor", step: int):
        if not (self.enabled and self.writer): return
        import torch
        x = img_hw
        if x.device.type != "cpu": x = x.cpu()
        x = x.float()
        if x.numel() > 0:
            m, M = x.min(), x.max()
            x = (x - m) / (M - m + 1e-8)
        x = x.unsqueeze(0)  # [1,H,W]
        self.writer.add_image(tag, x, step, dataformats='CHW')

    def add_histogram(self, tag: str, values: "torch.Tensor", step: int, bins: int = 50):
        if not (self.enabled and self.writer): return
        import torch
        v = values
        if isinstance(v, torch.Tensor):
            v = v.detach()
            if v.device.type != "cpu": v = v.cpu()
            v = v.reshape(-1).float()
        self.writer.add_histogram(tag, v, global_step=step, bins=bins)

    def add_text(self, tag: str, text: str, step: int):
        if not (self.enabled and self.writer): return
        self.writer.add_text(tag, text, step)

    def flush(self):
        if self.enabled and self.writer: self.writer.flush()

    def close(self):
        if self.enabled and self.writer: self.writer.close()


#### 4.2 Memory tracer & TensorBoard memory visualizer

In [39]:
from contextlib import contextmanager

class MemoryTracer:
    def __init__(self): self.frames = []
    def __call__(self, frame): self.frames.append(frame)
    def stack(self, key, bidx: int = 0):
        import torch
        return torch.stack([f[key][bidx] for f in self.frames], dim=0)  # [T, ...]

@contextmanager
def trace_memory(module):
    tracer = MemoryTracer()
    memories = [m for m in module.modules() if m.__class__.__name__ == "DNCMemory"]
    for m in memories: m.probe = tracer
    try:
        yield tracer
    finally:
        for m in memories: m.probe = None

@torch.no_grad()
def visualize_memory_tb(head, tok, writer, global_step: int, prompt="### Instruction:\nRemember A then B; later return A.\n\n### Response:\n", max_T=64):
    if writer is None: return
    enc = tok(prompt, return_tensors="pt").to(device)
    enc.input_ids = enc.input_ids[:, :max_T]
    from torch.amp import autocast
    with trace_memory(head) as tracer:
        with autocast('cuda', dtype=amp_dtype, enabled=(amp_dtype!=torch.float32)):
            _ = head(enc.input_ids)

    u = tracer.stack("u")           # [T, N]
    Mnorm = tracer.stack("M_norm")  # [T, N]
    rw = tracer.stack("rw")         # [T, R, N]

    # Log images
    writer.add_image("memory/u_TxN", (u.T).unsqueeze(0), global_step, dataformats='CHW')
    writer.add_image("memory/Mnorm_TxN", (Mnorm.T).unsqueeze(0), global_step, dataformats='CHW')

    top_read = rw.argmax(dim=-1).float()  # [T,R]
    top_read_img = (top_read / max(1, rw.size(-1)-1)).T  # [R,T]
    writer.add_image("memory/top_read_RxT", top_read_img.unsqueeze(0), global_step, dataformats='CHW')


## 5. Training

#### 5.1 Schedulers

In [40]:
# --- LR Scheduler: linear warmup -> cosine decay (nonzero start) ---

def make_warmup_cosine_scheduler(optimizer, warmup_steps: int, total_steps: int, min_lr_ratio: float = 0.10):
    """
    Warms up linearly from 0->1 over warmup_steps; cosine decays from 1->min_lr_ratio for the remainder.
    Uses step_idx+1 to avoid zero LR at the start.
    """
    warmup_steps = max(1, int(warmup_steps))
    total_steps = max(warmup_steps + 1, int(total_steps))

    def lr_lambda(step_idx: int):
        s = step_idx + 1
        if s <= warmup_steps:
            return s / float(warmup_steps)
        progress = (s - warmup_steps) / float(max(1, total_steps - warmup_steps))
        return max(min_lr_ratio, 0.5 * (1.0 + math.cos(math.pi * progress)))

    return LambdaLR(optimizer, lr_lambda)


#### 5.2 Training Utilities

In [41]:
from torch.amp import GradScaler, autocast

def build_model_and_tokenizer():
    tok, base = load_base_model(CFG.base_model_id)
    if CFG.d_model is None:
        CFG.d_model = base.config.hidden_size
    head = DNCFormerHead(base, CFG).to(device)
    if CFG.use_torch_compile and hasattr(torch, 'compile'):
        head = torch.compile(head)
    #print("Trainable params in head:", count_params(head))
    return tok, head

def make_optimizer(model):
    params = [p for p in model.parameters() if p.requires_grad]
    return torch.optim.AdamW(params, lr=CFG.lr, weight_decay=CFG.weight_decay)

def lm_shift_labels(input_ids, logits, tok):
    labels = input_ids[:, 1:].contiguous()
    logits = logits[:, :-1].contiguous()
    return F.cross_entropy(logits.view(-1, logits.size(-1)), labels.view(-1), ignore_index=tok.pad_token_id)

#### 5.3 Experimental training loop, stage 1 experiments

In [42]:
from torch.amp import GradScaler, autocast

def train_experiment(
    steps: int = None,
    batch_size: int = None,
    warmup_steps: int = None,
    min_lr_ratio: float = 0.1,
    mixture_weights=(0.4, 0.2, 0.2, 0.2),
    hf_dataset: str = "tatsu-lab/alpaca",
    hf_max_items: int = 5000,
    log_every: int = None,
    viz_memory_after: bool = False,
    viz_prompt: str = "### Instruction:\nSay hello in one word\n\n### Response:\n",
    viz_max_T: int = 64,
    # NEW: optional schedules (each a list of (until_step, value))
    mixture_schedule=None,         # e.g., [(100, (0.3,0.3,0.2,0.2)), (None, (0.4,0.2,0.2,0.2))]
    gate_temp_schedule=None,       # e.g., [(100, 0.8), (None, 1.0)]
    gate_reg_schedule=None,        # e.g., [(100, 3e-4), (None, 2e-4)]
):
    cfg = CFG
    steps = int(steps or cfg.train_steps)
    batch_size = int(batch_size or getattr(cfg, "batch_size", 8))
    warmup_steps = int(warmup_steps if warmup_steps is not None else getattr(cfg, "warmup_steps", max(10, steps // 20)))
    log_every = int(log_every if log_every is not None else getattr(cfg, "log_every", 10))

    # Model & tokenizer
    tok, head = build_model_and_tokenizer()
    optim = make_optimizer(head)
    scheduler = make_warmup_cosine_scheduler(optim, warmup_steps, steps, min_lr_ratio=min_lr_ratio)

    # Sampler (and allow schedules to change weights)
    mixer = _build_mixer(tok, mixture_weights, hf_dataset=hf_dataset, hf_max_items=hf_max_items)

    def _apply_schedules(step: int, mix_name_hint=None):
        # mixture schedule
        if mixture_schedule:
            for until, ws in mixture_schedule:
                if until is None or step <= int(until):
                    try:
                        mixer.set_weights(ws)
                    except Exception:
                        pass
                    break
        # gate temp schedule
        if gate_temp_schedule:
            for until, temp in gate_temp_schedule:
                if until is None or step <= int(until):
                    setattr(CFG, "gate_temp", float(temp))
                    break
        # gate reg schedule
        if gate_reg_schedule:
            for until, lam in gate_reg_schedule:
                if until is None or step <= int(until):
                    setattr(CFG, "gate_reg_lambda", float(lam))
                    break

    # TB setup
    global tb
    if TB_AVAILABLE:
        try:
            tb  # NameError if not defined
        except NameError:
            tb = TBLogger(logdir="./runs")
        if not isinstance(tb, TBLogger) or getattr(tb, "writer", None) is None:
            tb = TBLogger(logdir="./runs")
        tblog = True
        tb.add_text("run/config", json.dumps({
            "steps": steps,
            "batch_size": batch_size,
            "warmup_steps": warmup_steps,
            "mixture_weights": list(mixture_weights),
            "mixture_schedule": mixture_schedule,
            "gate_temp_schedule": gate_temp_schedule,
            "gate_reg_schedule": gate_reg_schedule,
        }, indent=2), 0)
    else:
        tblog = False

    head.train()
    scaler = torch.amp.GradScaler('cuda', enabled=(amp_dtype in (torch.float16, torch.bfloat16)))

    for step in range(1, steps + 1):
        _apply_schedules(step)

        in_ids = mixer(batch_size).to(device)
        with torch.autocast('cuda', dtype=amp_dtype, enabled=(amp_dtype != torch.float32)):
            logits, gates, aux = head.forward_with_metrics(in_ids, gate_override=getattr(CFG, "force_g", None))
            loss = lm_shift_labels(in_ids, logits, tok)

            # Optional: mild gate-usage regularizer (same as before)
            lam = float(getattr(CFG, "gate_reg_lambda", 0.0))
            if lam > 0 and isinstance(gates, (list, tuple)) and len(gates) > 0:
                reg = 0.0
                for g in gates:
                    g2 = _reduce_gate_tensor(g)
                    # Encourage decisive routing on memory batches only? For now, uniform
                    reg = reg + (g2.mean() * 0.0 + (g2 * (1 - g2)).mean())  # small entropy-like penalty
                loss = loss + lam * reg

        use_scaler = (amp_dtype in (torch.float16, torch.bfloat16))
        if use_scaler:
            scaler.scale(loss).backward()
            torch.nn.utils.clip_grad_norm_(head.parameters(), CFG.grad_clip)
            scaler.step(optim); scaler.update(); optim.zero_grad(set_to_none=True)
        else:
            loss.backward()
            torch.nn.utils.clip_grad_norm_(head.parameters(), CFG.grad_clip)
            optim.step(); optim.zero_grad(set_to_none=True)

        scheduler.step()

        # --- Logging (TB) ---
        if step % log_every == 0 and tblog:
            # global scalars
            tb.writer.add_scalar("train/loss", float(loss.item()), step)
            tb.writer.add_scalar("train/lr", float(scheduler.get_last_lr()[0]), step)

            # gate metrics per block (global)
            if isinstance(gates, (list, tuple)):
                for bi, g in enumerate(gates):
                    g_mean, g_frac, g_ent = _gate_metrics(g)
                    tb.writer.add_scalar(f"gates/block_{bi}_mean", g_mean, step)
                    tb.writer.add_scalar(f"gates/block_{bi}_frac>0.5", g_frac, step)
                    tb.writer.add_scalar(f"gates/block_{bi}_entropy", g_ent, step)

                # per-task metrics (using the sampler's last batch type)
                mix_name = getattr(mixer, "last_name", None) or "unknown"
                tb.writer.add_scalar(f"loss_by_task/{mix_name}", float(loss.item()), step)
                for bi, g in enumerate(gates):
                    g_mean, g_frac, g_ent = _gate_metrics(g)
                    tb.writer.add_scalar(f"gates_by_task/block_{bi}_mean/{mix_name}", g_mean, step)
                    tb.writer.add_scalar(f"gates_by_task/block_{bi}_frac>0.5/{mix_name}", g_frac, step)

                # optional: quartiles (block 0)
                if len(gates) > 0:
                    g0 = _reduce_gate_tensor(gates[0].detach())
                    T = g0.size(1); q = max(1, T // 4)
                    slices = [(0, q), (q, 2*q), (2*q, 3*q), (3*q, T)]
                    for qi, (s, e) in enumerate(slices, start=1):
                        tb.writer.add_scalar(f"gates/block0_q{qi}_mean/{mix_name}", float(g0[:, s:e].mean().item()), step)

            tb.flush()

        # --- Console echo (always) ---
        if step % log_every == 0:
            g_means_print = []
            if isinstance(gates, (list, tuple)):
                for g in gates:
                    g_means_print.append(float(_reduce_gate_tensor(g).mean().item()))
            print(f"step {step} | loss {loss.item():.4f} | lr {scheduler.get_last_lr()[0]:.2e} | "
                  f"gates={g_means_print} | mix={getattr(mixer,'last_name','?')}")

    # Optional memory viz
    if viz_memory_after:
        try:
            visualize_memory_tb(head, tok, tb.writer, global_step=steps, prompt=viz_prompt, max_T=viz_max_T)
        except Exception as e:
            print("Memory TB viz skipped:", e)
    
    # flush log      
    if tblog:
        tb.flush()

    return head, tok


#### 5.4 Robust forward for ParallelEnrichmentBlock
TODO: this is a monkeypatch, non-critical but fold in properly when I have time

In [43]:
# # --- Patch: robust forward for ParallelEnrichmentBlock ---
def _peb_forward(self, x, dnc_state=None, gate_override=None):
    """
    Returns: (out, dnc_state, g)
      out: (B,T,D), mixed from vanilla (vt) and DNC (dt)
      dnc_state: updated state dict from DNC path
      g: (B,T,D) or (B,T,1) gate values in [0,1]
    """
    # 1) Vanilla transformer path
    T = x.size(1)
    mask = causal_mask(T, device=x.device)  # (T,T) causal attn mask
    x_cast = x.to(self.vanilla.ln1.weight.dtype)
    vt = self.vanilla(x_cast, attn_mask=mask)    # (B,T,D)

    # 2) DNC path (controller+memory)
    dt, dnc_state = self.dncblock(x, state=dnc_state)  # dt: (B,T,D)

    # 3) Gate computation (optionally LN before gate)
    import torch as _t
    z = _t.cat([vt, dt], dim=-1)  # (B,T,2D)
    h = self.pre_gate_ln(z) if hasattr(self, "pre_gate_ln") and self.pre_gate_ln is not None else z
    g_pre = self.gate(h)                 # typically (B,T,D) or (B,T,1)
    tau = float(getattr(CFG, 'gate_temp', 1.0))
    g = torch.sigmoid(g_pre / max(tau, 1e-6))

    # Optional ablation override: force g to 0.0 (vanilla) or 1.0 (DNC)
    if gate_override is not None:
        g = _t.full_like(g, float(gate_override))

    # 4) Blend paths
    out = g * dt + (1.0 - g) * vt        # (B,T,D)

    # 5) Lightweight metrics (only if requested)
    self.last_metrics = None
    if getattr(self, "collect_metrics", False):
        try:
            eps = 1e-6
            gc = g.clamp(min=eps, max=1.0 - eps)
            g_entropy = (-(gc * gc.log() + (1 - gc) * (1 - gc).log())).mean()
            vt_norm = vt.norm(dim=-1).mean()
            dt_norm = dt.norm(dim=-1).mean()
            mstats = {}
            # Pull aggregated memory stats if present
            if isinstance(dnc_state, dict) and isinstance(dnc_state.get("stats", None), dict):
                for k in ("u_mean", "rw_max_mean", "ww_max_mean", "M_norm_mean"):
                    if k in dnc_state["stats"]:
                        mstats[k] = dnc_state["stats"][k]
            self.last_metrics = {
                "g_mean": g.mean().detach(),
                "g_entropy": g_entropy.detach(),
                "vt_norm": vt_norm.detach(),
                "dt_norm": dt_norm.detach(),
                **mstats,
            }
        except Exception:
            self.last_metrics = None

    return out, dnc_state, g

# Apply the patch
ParallelEnrichmentBlock.forward = _peb_forward


## 6. Eval harnesses

#### 6.1 copy, reverse, needle-in-haystack

In [44]:
from torch.amp import autocast

@torch.no_grad()
def eval_copy(head, tok, batch=4, T=64, vocab=100):
    x = make_copy_task(batch, T, vocab=vocab).to(device)
    with autocast('cuda', dtype=amp_dtype, enabled=(amp_dtype!=torch.float32)):
        logits, gates = head(x)
    preds = logits.argmax(dim=-1)
    acc = (preds[:, :-1] == x[:, 1:]).float().mean().item()
    print("copy acc:", acc, "gates:", [g.mean().item() for g in gates])
    return {"acc": acc, "gates": [g.mean().item() for g in gates]}

@torch.no_grad()
def eval_reverse(head, tok, batch=4, T=64, vocab=100):
    x, y = make_reverse_task(batch, T, vocab=vocab)
    x, y = x.to(device), y.to(device)
    with autocast('cuda', dtype=amp_dtype, enabled=(amp_dtype!=torch.float32)):
        logits, gates = head(x)
    preds = logits.argmax(dim=-1)
    acc = (preds[:, :-1] == y[:, 1:]).float().mean().item()
    print("reverse acc:", acc, "gates:", [g.mean().item() for g in gates])
    return {"acc": acc, "gates": [g.mean().item() for g in gates]}

@torch.no_grad() # WARNING - planned for depreciation
def eval_needle(head, tok, batch=4, T=128, vocab=200):
    x = make_needle_task(batch, T, vocab=vocab).to(device)
    with autocast('cuda', dtype=amp_dtype, enabled=(amp_dtype!=torch.float32)):
        logits, gates = head(x)
    preds = logits.argmax(dim=-1)
    acc = (preds[:, -1] == x[:, -1]).float().mean().item()
    print("needle acc:", acc, "gates:", [g.mean().item() for g in gates])
    return {"acc": acc, "gates": [g.mean().item() for g in gates]}

@torch.no_grad()
def evaluate_haystack(head, steps: int = 50, batch: int = 16, T: int = 256, vocab: int = 1024,
                      tb_step: int = None, fast: bool = False):
    """
    Long-span retrieval probe: ... K V ... K ? => predict V at '?'.
    fast=True uses inference_mode() and smaller defaults to speed up sweeps.
    """
    from torch.amp import autocast
    head.eval()

    # Fast path defaults (can still be overridden by explicit args)
    if fast:
        steps = min(steps, 10)
        batch = min(batch, 8)
        T = min(T, 128)

    device_ = next(head.parameters()).device
    use_amp = (amp_dtype in (torch.float16, torch.bfloat16)) and torch.cuda.is_available()

    accs, losses = [], []
    ctx = torch.inference_mode() if fast else torch.no_grad()
    with ctx:
        for _ in range(steps):
            x, V, qpos = make_haystack_batch(batch, T=T, vocab=vocab)
            x = x.to(device_); V = V.to(device_); qpos = qpos.to(device_)

            if use_amp:
                with autocast(device_type="cuda", dtype=amp_dtype):
                    logits, _g = head(x)
            else:
                logits, _g = head(x)

            idx = torch.arange(x.size(0), device=device_)
            logits_q = logits[idx, qpos, :].float()
            loss = F.cross_entropy(logits_q, V)
            pred = logits_q.argmax(dim=-1)

            accs.append((pred == V).float().mean().item())
            losses.append(loss.item())

    acc_m = float(np.mean(accs)); loss_m = float(np.mean(losses))

    if TB_AVAILABLE and ('tb' in globals()):
        tb.writer.add_scalar("eval/haystack_acc", acc_m, tb_step if tb_step is not None else 0)
        tb.writer.add_scalar("eval/haystack_loss", loss_m, tb_step if tb_step is not None else 0)
        tb.flush()

    head.train()
    print(f"[Haystack] acc={acc_m:.3f} | loss={loss_m:.3f} | fast={fast}")
    return acc_m, loss_m

## 7. Unit tests, smoke tests

#### 7.1 basic unit test, model integrity

In [45]:
def run_basic_unit_tests():
    B, T = 2, 4
    R, W, N = CFG.dnc_read_heads, CFG.dnc_cell_size, CFG.dnc_nr_cells
    d_in = d_model = 128

    mem = DNCMemory(N, W, R).to(device)
    state = mem.reset(B, device=device)
    iface = {
        "k_read": torch.zeros(B,R,W, device=device),
        "beta_read": torch.ones(B,R,1, device=device),
        "k_write": torch.zeros(B,W, device=device),
        "beta_write": torch.ones(B,1, device=device),
        "erase": torch.sigmoid(torch.randn(B,W, device=device)),
        "write_vec": torch.randn(B,W, device=device),
        "free_gates": torch.sigmoid(torch.randn(B,R,1, device=device)),
        "alloc_gate": torch.sigmoid(torch.randn(B,1, device=device)),
        "write_gate": torch.sigmoid(torch.randn(B,1, device=device)),
        "read_mode": torch.randn(B,R,3, device=device),
    }
    r, state2 = mem(iface, state)
    assert r.shape == (B,R,W)

    ctrl = TransformerController(d_in+R*W, d_model, heads=4).to(device)
    x = torch.randn(B,T,d_in, device=device)
    reads = state2["r"].reshape(B,R*W).unsqueeze(1).expand(B,T,R*W)
    h = ctrl(torch.cat([x, reads], dim=-1))
    assert h.shape == (B,T,d_model)

    dblk = DNCformerBlock(d_in, d_model, R, W, N, heads=4, dropout=0.1, ffn_mult=4.0).to(device)
    y, s = dblk(x)
    assert y.shape == (B,T,d_model)

    pen = ParallelEnrichmentBlock(d_model, d_in, R, W, N, heads=4, dropout=0.1, ffn_mult=4.0, gate_bias_init=-1.0).to(device)
    out, s2, g = pen(torch.randn(B,T,d_model, device=device))
    print(f"out: {out}\ns2: {s2}\ng: {g}")
    eps = 1e-6
    assert torch.isfinite(g).all()
    assert ((g > eps) & (g < 1 - eps)).float().mean().item() > 0.95
    assert out.shape == (B,T,d_model)
    print("All unit-like tests passed.")


In [46]:
#run_basic_unit_tests()

#### 7.2 Evaluator unit tests

In [47]:
import torch
from typing import Optional

class _DummyHead(torch.nn.Module):
    def __init__(self, vocab=100, d_model=64, n_blocks=2):
        super().__init__()
        self.vocab = vocab
        self.d_model = d_model
        self.n_blocks = n_blocks
    def forward(self, input_ids: torch.Tensor, attention_mask: Optional[torch.Tensor]=None):
        B, T = input_ids.shape
        logits = torch.randn(B, T, self.vocab, device=input_ids.device, dtype=torch.float32)
        gates = [torch.sigmoid(torch.randn(B, T, self.d_model, device=input_ids.device)) for _ in range(self.n_blocks)]
        return logits, gates

def run_eval_unit_tests():
    dummy = _DummyHead().to(device).eval()
    res_copy = eval_copy(dummy, tok=None, batch=2, T=8, vocab=50)
    res_rev = eval_reverse(dummy, tok=None, batch=2, T=8, vocab=50)
    res_needle = eval_needle(dummy, tok=None, batch=2, T=16, vocab=100)
    assert all(k in res_copy for k in ["acc","gates"])
    assert all(k in res_rev for k in ["acc","gates"])
    assert all(k in res_needle for k in ["acc","gates"])
    print("Evaluator unit tests passed.")


In [48]:
#run_eval_unit_tests()

#### 7.3 GPU VRAM diagnostics + cuda allocation smoke test

In [49]:
import torch, gc, contextlib

def cuda_report(tag=""):
    if not torch.cuda.is_available():
        print("[cuda_report] CUDA not available"); return
    free, total = torch.cuda.mem_get_info()
    alloc = torch.cuda.memory_allocated()
    reserv = torch.cuda.memory_reserved()
    print(f"[{tag}] alloc={alloc/1e9:.2f} GB | reserved={reserv/1e9:.2f} GB | free={free/1e9:.2f} GB | total={total/1e9:.2f} GB")

def list_head_refs():
    # looks for globals named 'head' and count of DNCFormerHead instances
    import gc, inspect, sys
    heads = [o for o in gc.get_objects() if o.__class__.__name__ == "DNCFormerHead"]
    print(f"[liveness] DNCFormerHead instances alive: {len(heads)}")
    if 'head' in globals():
        h = globals()['head']
        try:
            devs = sorted({p.device.type for p in h.parameters()})
        except Exception:
            devs = ["<unknown>"]
        print(f"[liveness] global 'head' present; param devices: {devs}")
    else:
        print("[liveness] no global 'head'")
        
from transformers import AutoModelForCausalLM, AutoTokenizer
gc.collect(); torch.cuda.empty_cache(); cuda_report("pre base-only")
tok = AutoTokenizer.from_pretrained(CFG.base_model_id, trust_remote_code=True)
base = AutoModelForCausalLM.from_pretrained(
    CFG.base_model_id,
    torch_dtype=(torch.bfloat16 if (amp_dtype!=torch.float32 and torch.cuda.is_bf16_supported()) else (torch.float16 if amp_dtype!=torch.float32 else None)),
    attn_implementation="sdpa",
    trust_remote_code=True,
    low_cpu_mem_usage=True,
    use_safetensors=True,
).to("cuda")
cuda_report("after base-only")
del base, tok; gc.collect(); torch.cuda.empty_cache()


[pre base-only] alloc=0.00 GB | reserved=0.00 GB | free=24.44 GB | total=25.77 GB


`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

[after base-only] alloc=7.64 GB | reserved=7.64 GB | free=16.79 GB | total=25.77 GB


#### 7.4 TB logger test

In [50]:
tb = TBLogger(logdir="./runs")
try:
    tb
except NameError:
    tb = TBLogger(logdir="./runs")
print("TB active:", TB_AVAILABLE, "| logdir:", getattr(tb, "path", None))

TB active: True | logdir: ./runs\dncformer-20250820-093245


#### 7.5 Memory tracer smoke test

In [51]:
def run_memory_tracer_smoke(head, tok):
    if not TB_AVAILABLE:
        print("TB not available; skipping image smoke.")
        return
    from torch.amp import autocast
    with trace_memory(head) as tracer:
        x = torch.randint(5, (1, 16), device=device)
        with autocast('cuda', dtype=amp_dtype, enabled=(amp_dtype!=torch.float32)):
            _ = head(x)
    assert len(tracer.frames) > 0, "No memory frames captured"
    print("Tracer captured", len(tracer.frames), "steps")


In [52]:
#run_memory_tracer_smoke()

#### 7.6 Data generator smoke test

In [53]:

mx = int(getattr(CFG, "max_seq_len", 256))
tok, _base = tok if 'tok' in globals() else (None, None)
_pad = getattr(tok, "pad_token_id", 0) if tok is not None else 0

x = make_repeat_copy(batch=3, T=min(mx, 128), vocab=50, pad_id=_pad)
print("repeat_copy batch shape:", x.shape, "| dtype:", x.dtype)  # expect (3, <=128), long
assert x.dtype == torch.long


repeat_copy batch shape: torch.Size([3, 128]) | dtype: torch.int64


#### 7.7 Mixture sampler smoke test

In [54]:
ms = MixtureSampler(gens=[lambda b: None, lambda b: None, lambda b: None, lambda b: None],
                    weights=[0.4,0.2,0.2,0.2],
                    names=["hf","copy","repeat","nback"])
print("p0:", ms.p.tolist())    # ~[0.4,0.2,0.2,0.2]
ms.set_weights([0.3,0.3,0.25,0.15])
print("p1:", ms.p.tolist())    # ~[0.3,0.3,0.25,0.15]

p0: [0.4000000059604645, 0.20000000298023224, 0.20000000298023224, 0.20000000298023224]
p1: [0.30000001192092896, 0.30000001192092896, 0.25, 0.15000000596046448]


#### 7.8 Build mixer smoke test

In [55]:
# 1) With Alpaca (instruction/output)
tok, _ = build_model_and_tokenizer()
m1 = _build_mixer(tok, (0.4,0.2,0.2,0.2), hf_dataset="tatsu-lab/alpaca", hf_max_items=500)
print("names:", m1.names, "| p:", m1.p.tolist())  # expect 'hf' present

# 2) With TinyStories (text)
m2 = _build_mixer(tok, (0.4,0.2,0.2,0.2), hf_dataset="roneneldan/TinyStories", hf_max_items=500)
print("names:", m2.names, "| p:", m2.p.tolist())  # expect 'hf' present (text path)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

names: ['hf', 'copy', 'repeat', 'nback'] | p: [0.4000000059604645, 0.20000000298023224, 0.20000000298023224, 0.20000000298023224]
names: ['hf', 'copy', 'repeat', 'nback'] | p: [0.4000000059604645, 0.20000000298023224, 0.20000000298023224, 0.20000000298023224]


#### 7.9 haystack smoke test

In [56]:
tok, head = build_model_and_tokenizer()
acc, loss = evaluate_haystack(head, steps=2, batch=4, T=64, vocab=512, tb_step=0)
print("Haystack smoke:", acc, loss)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

C:\Users\andre\miniconda3\envs\dncformer\lib\contextlib.py:103: FutureWarning: `torch.backends.cuda.sdp_kernel()` is deprecated. In the future, this context manager will be removed. Please see `torch.nn.attention.sdpa_kernel()` for the new context manager, with updated signature.
  self.gen = func(*args, **kwds)
You are not running the flash-attention implementation, expect numerical differences.


[Haystack] acc=0.000 | loss=10.112 | fast=False
Haystack smoke: 0.0 10.111836910247803


#### 7.10 Training run Sanity test

In [57]:
free_head_and_cache()
cuda_report("snapshot: before train_experiment")
head, tok = train_experiment(steps=10, warmup_steps=10, mixture_weights=(0.4,0.2,0.2,0.2))
cuda_report("snapshot: after train_experiment")
#Launch TensorBoard in a terminal: tensorboard --logdir ./runs


[after free] alloc=9.78 GB | reserved=9.81 GB | free=14.57 GB | total=25.77 GB
[snapshot: before train_experiment] alloc=9.78 GB | reserved=9.81 GB | free=14.57 GB | total=25.77 GB


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

step 10 | loss 3.5799 | lr 2.00e-05 | gates=[0.283203125, 0.275390625] | mix=hf
[snapshot: after train_experiment] alloc=19.56 GB | reserved=33.25 GB | free=0.00 GB | total=25.77 GB


## 8. Experiments - Stage 1 - basic architecture

#### 8.1 Experiment Set 1 - Medium training sweep, testing parameter / dataset variants
 - E0: Baseline
 - E1: memory/algorithmic slanted data mix
 - E2: gate regularizer (low)
 - E3: gate regularizer (high)
 - E4: sharper routing via lower gate temperature
 - E5: ablations: disable/force DNC path
    - 5a: disable DNC path (DNC path disabled)
    - 5b: force DNC path (transformer path disabled) 

In [ ]:
import time, torch, gc

# run set parameters
EXP_STEPS   = 500       # try 1000–5000 for longer curves
EXP_WARMUP  = 10       # keep ~steps/20
BASE_MIX    = (0.4, 0.2, 0.2, 0.2)  # (hf, copy, repeat, nback)

def set_cfg(**kv):
    for k, v in kv.items():
        setattr(CFG, k, v)

def run_isolate_test(label,
            mixture_weights=BASE_MIX,
            gate_reg_lambda=None,
            gate_temp=None,
            force_g=None,
            steps=EXP_STEPS,
            warmup=EXP_WARMUP):
    """Run a single train_experiment experiment with explicit knobs."""
    print(f"\n=== {label} ===")
    # set experiment-specific knobs (others come from CFG defaults)
    if gate_reg_lambda is not None: set_cfg(gate_reg_lambda=float(gate_reg_lambda))
    if gate_temp is not None:       set_cfg(gate_temp=float(gate_temp))
    set_cfg(force_g=force_g)  # may be None, 0.0, or 1.0

    print(f"mixture={mixture_weights}, gate_reg_lambda={getattr(CFG,'gate_reg_lambda', 0.0)}, "
          f"gate_temp={getattr(CFG,'gate_temp', 1.0)}, force_g={getattr(CFG,'force_g', None)}")

    # clean slate for VRAM and allocator fragmentation between runs
    free_head_and_cache()
    cuda_report(f"before {label}")
    time.sleep(1.2)  # ensure distinct TB run dirs (timestamp granularity)

    # run
    head, tok = train_experiment(
        steps=steps,
        warmup_steps=warmup,
        mixture_weights=mixture_weights,
        viz_memory_after=False,   # keep quick; use visualize_memory_tb() ad-hoc
    )

    # post-run snapshot + cleanup
    cuda_report(f"after  {label}")
    free_head_and_cache()
    return head, tok

# === E0 baseline: identical to your sanity run ===
run_isolate_test("E0_baseline",
        mixture_weights=(0.4, 0.2, 0.2, 0.2),
        gate_reg_lambda=0.0,
        gate_temp=1.0,
        force_g=None,
        steps=EXP_STEPS, warmup=EXP_WARMUP)

# === E1 memory-leaning mix: more algorithmic exposure ===
run_isolate_test("E1_memory_lean",
        mixture_weights=(0.4, 0.3, 0.2, 0.1),  # HF, copy, repeat, nback
        gate_reg_lambda=0.0,
        gate_temp=1.0,
        force_g=None,
        steps=EXP_STEPS, warmup=EXP_WARMUP)

# === E2 gate regularizer (low) ===
run_isolate_test("E2_gate_reg_low",
        mixture_weights=(0.4, 0.2, 0.2, 0.2),
        gate_reg_lambda=2e-4,
        gate_temp=1.0,
        force_g=None,
        steps=EXP_STEPS, warmup=EXP_WARMUP)

# === E3 gate regularizer (high) ===
run_isolate_test("E3_gate_reg_high",
        mixture_weights=(0.4, 0.2, 0.2, 0.2),
        gate_reg_lambda=5e-4,
        gate_temp=1.0,
        force_g=None,
        steps=EXP_STEPS, warmup=EXP_WARMUP)

# === E4 sharper routing via lower gate temperature ===
run_isolate_test("E4_gate_temp_0p7",
        mixture_weights=(0.4, 0.2, 0.2, 0.2),
        gate_reg_lambda=0.0,
        gate_temp=0.7,
        force_g=None,
        steps=EXP_STEPS, warmup=EXP_WARMUP)

# === E5 ablations: disable/force DNC path ===
run_isolate_test("E5a_force_g_0",
        mixture_weights=(0.4, 0.2, 0.2, 0.2),
        gate_reg_lambda=0.0,
        gate_temp=1.0,
        force_g=0.0,
        steps=EXP_STEPS, warmup=EXP_WARMUP)

run_isolate_test("E5b_force_g_1",
        mixture_weights=(0.4, 0.2, 0.2, 0.2),
        gate_reg_lambda=0.0,
        gate_temp=1.0,
        force_g=1.0,
        steps=EXP_STEPS, warmup=EXP_WARMUP)



=== E0_baseline ===
mixture=(0.4, 0.2, 0.2, 0.2), gate_reg_lambda=0.0, gate_temp=1.0, force_g=None
[after free] alloc=9.79 GB | reserved=9.82 GB | free=14.54 GB | total=25.77 GB
[before E0_baseline] alloc=9.79 GB | reserved=9.82 GB | free=14.54 GB | total=25.77 GB


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
free_head_and_cache()

#### 8.2 Experiment Set 2 - parameter sweeps, based on E2 params from set 1 above
 - E6: higher gate temp
 - E7: memory-leaning warm-start
 - E8: capacity sweep
 - E9: baseline training, haystack eval

In [ ]:
import re, time

def set_cfg(**kv):
    for k, v in kv.items():
        setattr(CFG, k, v)
    
def start_tb_run(label: str = None, logdir: str = "./runs"):
    """Close any existing TB writer and open a fresh run dir with timestamp + optional label."""
    if not TB_AVAILABLE:
        print("TensorBoard not available; skipping start_tb_run.")
        return False
    global tb
    # Close/flush a previous writer if present
    try:
        if 'tb' in globals() and isinstance(tb, TBLogger) and getattr(tb, "writer", None):
            tb.flush(); tb.close()
    except Exception:
        pass

    ts = time.strftime("dncformer-%Y%m%d-%H%M%S")
    run_name = ts
    if label:
        safe = re.sub(r"[^A-Za-z0-9_.-]+", "_", str(label))
        run_name = f"{ts}-{safe}"

    tb = TBLogger(logdir=logdir, run_name=run_name)
    tb.add_text("run/label", str(label or "unlabeled"), 0)
    print("TB run started:", getattr(tb, "path", None))
    return True


def run_one_labeled(label, steps, mixture_weights, seed=1234,
                    mixture_schedule=None, gate_temp_schedule=None, gate_reg_schedule=None,
                    post_haystack=False):
    print(f"\n=== {label} | seed={seed} ===")
    random.seed(seed); np.random.seed(seed); torch.manual_seed(seed) 
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

    set_cfg(force_g=None)  # ensure no ablation
    # unique TB run for each experiment label
    start_tb_run(label)

    # echo run metadata
    if TB_AVAILABLE and 'tb' in globals():
        import json
        tb.add_text("run/meta", json.dumps({
            "label": label,
            "steps": steps,
            "mixture_weights": list(mixture_weights),
            "mixture_schedule": mixture_schedule,
            "gate_temp_schedule": gate_temp_schedule,
            "gate_reg_schedule": gate_reg_schedule,
        }, indent=2), 0)
    
    # print config stub
    print("CFG.gate_temp:", getattr(CFG, "gate_temp", 1.0),
          "| CFG.gate_reg_lambda:", getattr(CFG, "gate_reg_lambda", 0.0),
          "| mixture:", mixture_weights)

    free_head_and_cache()
    if 'cuda_report' in globals(): cuda_report(f"before {label}")

    head, tok = train_experiment(
        steps=steps,
        warmup_steps=max(10, steps//20),
        mixture_weights=mixture_weights,
        mixture_schedule=mixture_schedule,
        gate_temp_schedule=gate_temp_schedule,
        gate_reg_schedule=gate_reg_schedule,
        viz_memory_after=False,
    )

    if post_haystack:
        evaluate_haystack(head, steps=50, batch=16, T=256, vocab=1024, tb_step=steps, fast=True)

    if 'cuda_report' in globals(): cuda_report(f"after  {label}")
    free_head_and_cache()
    return head, tok

# Common settings
EXP_STEPS = 500
BASE_MIX  = (0.4, 0.2, 0.2, 0.2)
SEEDS     = [1337, 2027, 4242]

# === E6: gate_temp=0.8 (3 seeds), otherwise baseline ===
set_cfg(gate_reg_lambda=getattr(CFG, "gate_reg_lambda", 2e-4))  # low-λ default
for s in SEEDS:
    run_one_labeled(f"E6_temp0p8_seed{s}", steps=EXP_STEPS, mixture_weights=BASE_MIX,
                    seed=s,
                    gate_temp_schedule=[(None, 0.8)])

# === E7: memory-leaning warm-start (first 10% steps), then baseline; 3 seeds ===
warm_steps = max(50, EXP_STEPS // 10)
mix_warm   = (0.3, 0.3, 0.25, 0.15)  # a bit more memory-heavy than baseline
mix_main   = BASE_MIX
for s in SEEDS:
    run_one_labeled(f"E7_warmstart_seed{s}", steps=EXP_STEPS, mixture_weights=BASE_MIX,
                    seed=s,
                    mixture_schedule=[(warm_steps, mix_warm), (None, mix_main)],
                    gate_temp_schedule=[(warm_steps, 0.8), (None, 1.0)],
                    gate_reg_schedule=[(warm_steps, max(2e-4, getattr(CFG,'gate_reg_lambda', 2e-4))), (None, getattr(CFG,'gate_reg_lambda', 2e-4))])

# === E8: capacity sweep N=64 vs N=128 (1 seed each) ===
for N_val in (64, 128):
    set_cfg(N=N_val)  # assumes your DNC block reads CFG.N at construction
    run_one_labeled(f"E8_capacity_N{N_val}", steps=EXP_STEPS, mixture_weights=BASE_MIX,
                    seed=777,
                    gate_temp_schedule=[(None, getattr(CFG, 'gate_temp', 1.0))])

# Reset N if changed
set_cfg(N=getattr(CFG, 'N', 128))

# === E9: baseline with haystack eval ===
run_one_labeled("E9_baseline_haystack", steps=EXP_STEPS, mixture_weights=BASE_MIX,
                seed=31415, post_haystack=True)


In [ ]:
free_head_and_cache()

## 9. Experiments - Stage 2 - tiered/parallel memory systems

## 00. Misc.

#### Tensorboard log dump

In [ ]:
# ===== TensorBoard event analyzer for DNCFormer runs (robust) =====
# Discovers TB runs, merges multiple event files per run, summarizes, and exports granular CSVs.

import os, re, math, json, time, glob
from pathlib import Path
from collections import defaultdict

import numpy as np
import pandas as pd
from IPython.display import display

# --- TensorBoard imports (version-agnostic handling) ---
try:
    import tensorboard as _tb
    print("TensorBoard version:", getattr(_tb, "__version__", "unknown"))
except Exception as _e:
    print("TensorBoard import note:", _e)

try:
    from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
    from tensorboard.backend.event_processing import event_accumulator as ea_mod
except Exception as e:
    raise RuntimeError("TensorBoard not installed. Install via: pip install tensorboard") from e


# ---------------- helpers ----------------
def _size_guidance_version_safe():
    """Build size_guidance dict usable across TB versions."""
    sg = {}
    keys = ["SCALARS", "HISTOGRAMS", "IMAGES", "COMPRESSED_HISTOGRAMS", "AUDIO", "TENSORS"]
    for k in keys:
        v = getattr(ea_mod, k, None)
        if v is not None:
            sg[v] = 0
        else:
            sg[k.lower()] = 0
    return sg


def _infer_label_from_run_dir(run_dir: Path) -> str:
    """
    Expect 'dncformer-YYYYMMDD-HHMMSS-<LABEL>' or just 'dncformer-YYYYMMDD-HHMMSS'.
    Returns <LABEL> if present, else the directory name.
    """
    name = run_dir.name
    m = re.match(r".*-\d{8}-\d{6}-(.+)$", name)
    if m:
        return m.group(1)
    return name


def _load_scalars_from_event_file(ev_path: str) -> dict:
    """Load scalars from a single event file: tag -> list[(step, value)]."""
    acc = EventAccumulator(ev_path, size_guidance=_size_guidance_version_safe())
    acc.Reload()
    tags = acc.Tags().get('scalars', []) or []
    out = {}
    for tag in tags:
        vals = acc.Scalars(tag)
        out[tag] = [(int(x.step), float(x.value)) for x in vals]
    return out


def _merge_scalar_dicts(list_of_scalar_dicts):
    """
    Merge multiple event files for the same run.
    For each tag: keep the last value seen per (step), then return sorted by step.
    """
    merged = defaultdict(dict)  # tag -> {step: value}
    for scal in list_of_scalar_dicts:
        for tag, series in scal.items():
            d = merged[tag]
            for step, val in series:
                d[step] = val  # 'last wins' is fine; event files are append-only per run
    # Convert to tag -> sorted list[(step, value)]
    out = {}
    for tag, d in merged.items():
        steps_sorted = sorted(d.keys())
        out[tag] = [(s, d[s]) for s in steps_sorted]
    return out


def _detect_tasks_and_blocks(scalars: dict):
    """Detect task names and block ids from present tags. Returns (sorted_tasks, sorted_blocks)."""
    tasks = set()
    blocks = set()

    # tasks from "loss_by_task/<task>"
    for tag in scalars.keys():
        if tag.startswith("loss_by_task/"):
            tasks.add(tag.split("/", 1)[1])

    # blocks from "gates_by_task/block_<b>_mean/<task>" or "gates/block_<b>_mean"
    for tag in scalars.keys():
        m = re.match(r"gates_by_task/block_(\d+)_", tag)
        if m:
            blocks.add(int(m.group(1)))
        m2 = re.match(r"gates/block_(\d+)_mean$", tag)
        if m2:
            blocks.add(int(m2.group(1)))

    # sensible defaults if nothing is found
    if not tasks:
        tasks = {"hf", "copy", "repeat", "nback"}
    if not blocks:
        blocks = {0, 1}

    return sorted(tasks), sorted(blocks)


def s_last(vals, k=None):
    if not vals: return np.nan
    arr = np.array([v for _, v in vals], dtype=float)
    if k is None or k >= len(arr): return float(arr[-1])
    return float(np.nanmean(arr[-k:]))


def s_first(vals, k=None):
    if not vals: return np.nan
    arr = np.array([v for _, v in vals], dtype=float)
    if k is None or k >= len(arr): return float(arr[0])
    return float(np.nanmean(arr[:k]))


def s_mean(vals):
    if not vals: return np.nan
    return float(np.nanmean([v for _, v in vals]))


def s_count(vals):
    return len(vals) if vals else 0


# ---------------- discover runs ----------------
# Option 1: auto-discover all event files and group by their parent directory
FOUND_EVENTS = sorted(glob.glob("runs/**/events.out.tfevents.*", recursive=True))

# Option 2: pin a subset manually
# FOUND_EVENTS = [
#     r"runs/dncformer-20250819-122754-E6_gate_temp_0p8/events.out.tfevents....",
# ]

assert FOUND_EVENTS, "No event files found under ./runs. Have you executed any experiments?"

# Group event files by run directory
run_groups = defaultdict(list)   # run_dir_path -> [event_file_paths]
for p in FOUND_EVENTS:
    run_groups[str(Path(p).parent)].append(p)

print(f"Discovered {len(run_groups)} run(s).")


# ---------------- summarize each run ----------------
run_summaries = []
per_run_scalars = {}   # run_dir_name -> merged scalars dict

for run_dir, files in sorted(run_groups.items()):
    run_dir_path = Path(run_dir)
    label = _infer_label_from_run_dir(run_dir_path)
    run_id = run_dir_path.name

    try:
        # load and merge all files for this run
        scal_dicts = [_load_scalars_from_event_file(f) for f in sorted(files)]
        scal = _merge_scalar_dicts(scal_dicts)
        per_run_scalars[run_dir_path.name] = scal

        # detect tasks and blocks present
        TASKS, BLOCKS = _detect_tasks_and_blocks(scal)

        # basics
        loss_series = scal.get("train/loss", [])
        lr_series   = scal.get("train/lr", [])
        steps_logged = max([s for s, _ in loss_series], default=np.nan) if loss_series else np.nan
        loss0   = s_first(loss_series, k=5)
        lossT   = s_last(loss_series,  k=10)
        ldelta  = (loss0 - lossT) if not any(map(math.isnan, [loss0, lossT])) else np.nan
        lr_last = s_last(lr_series, k=1)

        # gates (global)
        g_means, g_entropy, g_frac_avg = {}, {}, {}
        for b in BLOCKS:
            g_means[b]   = s_last(scal.get(f"gates/block_{b}_mean", []), k=10)
            g_entropy[b] = s_last(scal.get(f"gates/block_{b}_entropy", []), k=10)
            fracs = []
            for t in TASKS:
                tag = f"gates_by_task/block_{b}_frac>0.5/{t}"
                if tag in scal:
                    fracs.append(s_last(scal[tag], k=10))
            g_frac_avg[b] = float(np.nanmean(fracs)) if fracs else np.nan

        # per-task losses (mean of last quarter of points for that task)
        task_loss_last, task_counts = {}, {}
        for t in TASKS:
            ts = scal.get(f"loss_by_task/{t}", [])
            task_counts[t] = s_count(ts)
            if ts:
                k = max(1, len(ts)//4)
                task_loss_last[t] = s_last(ts, k=k)
            else:
                task_loss_last[t] = np.nan

        # per-task gate means (avg last quarter)
        task_gmeans = {t: {} for t in TASKS}
        for t in TASKS:
            for b in BLOCKS:
                ts = scal.get(f"gates_by_task/block_{b}_mean/{t}", [])
                if ts:
                    k = max(1, len(ts)//4)
                    task_gmeans[t][b] = s_last(ts, k=k)
                else:
                    task_gmeans[t][b] = np.nan

        # quartiles (block0) if present
        q_means = {}
        for qi in range(1, 5):
            vals = []
            # May be logged globally or by task; check both
            tag_global = f"gates/block0_q{qi}_mean"
            if tag_global in scal:
                vals.append(s_last(scal[tag_global], k=10))
            else:
                for t in TASKS:
                    tag_task = f"gates/block0_q{qi}_mean/{t}"
                    if tag_task in scal:
                        vals.append(s_last(scal[tag_task], k=10))
            q_means[qi] = float(np.nanmean(vals)) if vals else np.nan

        # haystack eval if present
        hay_acc  = s_last(scal.get("eval/haystack_acc",  []), k=1)
        hay_loss = s_last(scal.get("eval/haystack_loss", []), k=1)

        # forced-g guess heuristic
        forced_guess = None
        gm_all = [g_means[b] for b in g_means if not math.isnan(g_means[b])]
        if gm_all:
            m = float(np.nanmean(gm_all))
            if m < 0.02:   forced_guess = "force_g=0"
            elif m > 0.98: forced_guess = "force_g=1"

        summary = {
            "label": label,
            "run_id": run_id,
            "run_dir": str(run_dir_path),
            "n_event_files": len(files),
            "steps_logged": steps_logged,
            "loss_start~5": loss0,
            "loss_end~10":  lossT,
            "loss_delta":   ldelta,
            "lr_last":      lr_last,
            "haystack_acc_last":  hay_acc,
            "haystack_loss_last": hay_loss,
            "forced_guess": forced_guess,
        }

        # flatten gate summaries
        for b in BLOCKS:
            summary[f"g_mean_b{b}"]     = g_means.get(b, np.nan)
            summary[f"g_entropy_b{b}"]  = g_entropy.get(b, np.nan)
            summary[f"g_frac>0.5_b{b}"] = g_frac_avg.get(b, np.nan)

        # flatten per-task last losses and per-task mean gates
        for t in TASKS:
            summary[f"loss_{t}_last"] = task_loss_last[t]
            for b in BLOCKS:
                summary[f"gmean_b{b}_{t}"] = task_gmeans[t][b]

        # quartiles
        for qi in range(1, 5):
            summary[f"g_b0_Q{qi}_mean"] = q_means[qi]

        run_summaries.append(summary)

    except Exception as e:
        print(f"[analyzer] Skipped run {run_dir}: {e}")

# -------- assemble run-level summary --------
df_runs = pd.DataFrame(run_summaries)
if df_runs.empty:
    print("No runs summarized.")
else:
    # Sort by timestamp embedded in run_id if possible
    def _ts_key(name: str):
        m = re.search(r"(\d{8})-(\d{6})", name or "")
        return (m.group(1), m.group(2)) if m else ("", "")
    df_runs = df_runs.sort_values(by=["run_id"], key=lambda s: s.map(_ts_key), ignore_index=True)

    # Arrange prominent columns
    front_cols = [c for c in [
        "label","run_id","n_event_files","steps_logged",
        "loss_start~5","loss_end~10","loss_delta","lr_last",
        "haystack_acc_last","haystack_loss_last","forced_guess",
        "g_mean_b0","g_mean_b1","g_entropy_b0","g_entropy_b1",
        "g_frac>0.5_b0","g_frac>0.5_b1",
        "loss_hf_last","loss_copy_last","loss_repeat_last","loss_nback_last",
        "gmean_b0_hf","gmean_b1_hf","gmean_b0_copy","gmean_b1_copy",
        "gmean_b0_repeat","gmean_b1_repeat","gmean_b0_nback","gmean_b1_nback",
        "g_b0_Q1_mean","g_b0_Q2_mean","g_b0_Q3_mean","g_b0_Q4_mean",
        "run_dir"
    ] if c in df_runs.columns]
    df_runs = df_runs[[*front_cols, *[c for c in df_runs.columns if c not in front_cols]]]

    display(df_runs)
    out_dir = Path("./analysis"); out_dir.mkdir(parents=True, exist_ok=True)
    df_runs.to_csv(out_dir / "run_level_summary.csv", index=False)
    print("Saved:", out_dir / "run_level_summary.csv")


# -------- granular per-task time series export --------
rows = []
for run_dir_name, scal in per_run_scalars.items():
    label = _infer_label_from_run_dir(Path(run_dir_name))
    # detect tasks/blocks present in this run
    TASKS, BLOCKS = _detect_tasks_and_blocks(scal)

    # pick up LR series for convenient join
    lr_by_step = {int(s): float(v) for s, v in scal.get("train/lr", [])}

    # per-task loss and per-task gate metrics
    for t in TASKS:
        loss_series = scal.get(f"loss_by_task/{t}", [])
        if not loss_series:
            continue

        gmean_by_step = {b: {int(s): float(v) for s, v in scal.get(f"gates_by_task/block_{b}_mean/{t}", [])}
                         for b in BLOCKS}
        gfrac_by_step = {b: {int(s): float(v) for s, v in scal.get(f"gates_by_task/block_{b}_frac>0.5/{t}", [])}
                         for b in BLOCKS}

        # Optional quartile logs (block 0)
        q_by_step = {qi: {int(s): float(v) for s, v in scal.get(f"gates/block0_q{qi}_mean/{t}", [])}
                     for qi in (1,2,3,4)}

        for step, loss_val in loss_series:
            step = int(step); loss_val = float(loss_val)
            for b in BLOCKS:
                row = {
                    "label": label,
                    "run_id": run_dir_name,
                    "task": t,
                    "block": b,
                    "step": step,
                    "loss": loss_val,
                    "g_mean": gmean_by_step[b].get(step, np.nan),
                    "g_frac>0.5": gfrac_by_step[b].get(step, np.nan),
                    "lr": lr_by_step.get(step, np.nan),
                }
                if b == 0:
                    for qi in (1,2,3,4):
                        row[f"g_b0_Q{qi}"] = q_by_step[qi].get(step, np.nan)
                rows.append(row)

df_task_ts = pd.DataFrame(rows)
if df_task_ts.empty:
    print("No per-task series found.")
else:
    df_task_ts = df_task_ts.sort_values(["label","task","step","block"], ignore_index=True)
    display(df_task_ts.head(20))
    out_dir = Path("./analysis"); out_dir.mkdir(parents=True, exist_ok=True)
    df_task_ts.to_csv(out_dir / "per_task_metrics.csv", index=False)
    print("Saved:", out_dir / "per_task_metrics.csv")


In [ ]:
# --- Rename ./runs/* so directory names include the experiment label (from TB text tags) ---
import os, re, glob, time, shutil
from pathlib import Path
from typing import Optional

# Try to close any active writer so files aren't locked on Windows
try:
    if 'tb' in globals() and getattr(tb, "writer", None) is not None:
        tb.flush(); tb.close()
        print("[rename] Closed active TB writer before renaming.")
except Exception as _e:
    print("[rename] Writer close note:", _e)

# TensorBoard event loading (version-agnostic size_guidance)
try:
    from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
    from tensorboard.backend.event_processing import event_accumulator as ea_mod
    try:
        from tensorboard.util import tensor_util as _tb_tensor_util
    except Exception:
        print("failed to load tensorboard utilities, but you'll install tensorflow on this system over my cold dead digital body."
              "\n\nfix your tensorboard installation and try again")
        _tb_tensor_util = None
except Exception as e:
    raise RuntimeError("TensorBoard not installed. `pip install tensorboard`") from e

def _size_guidance_version_safe():
    sg = {}
    for k in ["SCALARS","HISTOGRAMS","IMAGES","COMPRESSED_HISTOGRAMS","AUDIO","TENSORS"]:
        v = getattr(ea_mod, k, None)
        if v is not None:
            sg[v] = 0
        else:
            sg[k.lower()] = 0
    return sg

def _decode_text_tensor(tensor_proto) -> Optional[str]:
    """Decode TB text summary payload from a TensorEvent.tensor_proto."""
    try:
        if _tb_tensor_util is not None:
            arr = _tb_tensor_util.make_ndarray(tensor_proto)
        else:
            # very old stub fallback
            arr = _tb_make_ndarray(tensor_proto)
        val = arr.item() if arr.size == 1 else arr
        if isinstance(val, bytes):
            return val.decode("utf-8", "replace")
        if isinstance(val, str):
            return val
        # Some TB builds wrap a bytes array inside a 2D array
        if hasattr(val, "dtype") and str(val.dtype).startswith("|S"):
            return val.tobytes().decode("utf-8", "replace")
    except Exception:
        pass
    return None

def _infer_label_from_text_tags(run_dir: Path) -> Optional[str]:
    """Try run/label first, then run/meta (JSON with a 'label' field), then None."""
    ev_files = sorted(glob.glob(str(run_dir / "events.out.tfevents.*")))
    for ev in reversed(ev_files):
        try:
            acc = EventAccumulator(ev, size_guidance=_size_guidance_version_safe())
            acc.Reload()
            tags_t = acc.Tags().get("tensors", []) or []

            # 1) Preferred: run/label
            if "run/label" in tags_t:
                tens = acc.Tensors("run/label")
                for e in reversed(tens):
                    txt = _decode_text_tensor(e.tensor_proto)
                    if txt:
                        return txt.strip()

            # 2) Fallback: run/meta (JSON with label)
            if "run/meta" in tags_t:
                tens = acc.Tensors("run/meta")
                for e in reversed(tens):
                    txt = _decode_text_tensor(e.tensor_proto)
                    if txt:
                        txt = txt.strip()
                        # Sometimes add_text wraps in small HTML; tolerate raw JSON and simple strings
                        m = re.search(r"\{.*\}", txt, flags=re.S)
                        if m:
                            import json
                            try:
                                meta = json.loads(m.group(0))
                                if isinstance(meta, dict) and "label" in meta and meta["label"]:
                                    return str(meta["label"]).strip()
                            except Exception:
                                pass
                        # If it's just a string, return it
                        if txt and txt[0] not in "{<":
                            return txt
        except Exception:
            continue
    return None

def _infer_label_from_dirname(run_dir: Path) -> Optional[str]:
    """If the dir already has a '-<label>' suffix after timestamp, return that label."""
    m = re.match(r".*-\d{8}-\d{6}-(.+)$", run_dir.name)
    return m.group(1) if m else None

def _base_prefix(run_dir: Path) -> str:
    """Return 'dncformer-YYYYMMDD-HHMMSS' part if present, else the full name."""
    m = re.match(r"(.*-\d{8}-\d{6})(?:-.+)?$", run_dir.name)
    return m.group(1) if m else run_dir.name

def _slugify(s: str) -> str:
    return re.sub(r"[^A-Za-z0-9_.-]+", "_", s.strip())[:80] or "unlabeled"

def _is_active_run(run_dir: Path, seconds: int = 60) -> bool:
    """Heuristic: if any event file mtime is within the last `seconds`."""
    now = time.time()
    for ev in glob.glob(str(run_dir / "events.out.tfevents.*")):
        try:
            if now - os.path.getmtime(ev) < seconds:
                return True
        except Exception:
            pass
    return False

def rename_runs_by_label(log_root: str = "./runs", dry_run: bool = True,
                         skip_active_secs: int = 60) -> list[tuple[str, str]]:
    """
    Rename run directories under `log_root` to include label suffix (from TB text tag).
    Returns list of (old_path, new_path) actually renamed.
    """
    log_root = Path(log_root)
    renamed = []

    for run_dir in sorted([p for p in log_root.iterdir() if p.is_dir()]):
        # Already labeled?
        existing_label = _infer_label_from_dirname(run_dir)
        # Attempt tag-based label
        tag_label = _infer_label_from_text_tags(run_dir)

        label = tag_label or existing_label or "unlabeled"
        label_slug = _slugify(label)

        base = _base_prefix(run_dir)
        target = log_root / f"{base}-{label_slug}"

        # Skip if it's already the desired name
        if run_dir == target:
            print(f"[rename] OK (already labeled): {run_dir.name}")
            continue

        # Skip if run appears active
        if skip_active_secs and _is_active_run(run_dir, skip_active_secs):
            print(f"[rename] SKIP active (mtime<{skip_active_secs}s): {run_dir.name}")
            continue

        # Avoid collisions: add -v2, -v3, ...
        cand = target
        k = 2
        while cand.exists():
            cand = log_root / f"{base}-{label_slug}-v{k}"
            k += 1

        print(f"[rename] {run_dir.name}  ->  {cand.name}")
        if not dry_run:
            try:
                run_dir.replace(cand)
                renamed.append((str(run_dir), str(cand)))
            except Exception as e:
                print(f"[rename] FAILED: {run_dir} -> {cand}: {e}")

    return renamed

# --- Usage examples ---
# 1) Dry run (see planned changes)
_ = rename_runs_by_label("./runs", dry_run=True, skip_active_secs=60)

# 2) Execute renames
# _ = rename_runs_by_label("./runs", dry_run=False, skip_active_secs=60)
# print("Renamed:", _)


#### Notes & TODO
- The DNC memory here is **compact** and intended for research iteration; you can swap in a fuller reference implementation if desired. [x]
- The controller currently runs in **sequence mode** with a causal mask
- Training uses a tiny **instruction-following set** plus synthetic memory tasks.
- Gating is **vector-valued** with bias init favoring the vanilla path; metrics log mean gate values.
- Use `CFG.n_blocks` to grow the enrichment depth as VRAM allows.
